<a href="https://colab.research.google.com/github/BhaveshGoswami11/Generative-AI/blob/main/RAG_Chatbot_MS_ACS_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1) Introduction & Objectives

## What is RAG (Retrieval-Augmented Generation)?

**RAG** is a technique that combines **retrieval of relevant information** with **generation of text** using a large language model (LLM) like GPT. Instead of relying solely on the knowledge encoded in the model, RAG allows the model to **access external sources dynamically** to provide more accurate, up-to-date, or domain-specific answers.  

**How it works (high-level):**  
1. **Retrieve:** When a user asks a question, the system searches a **knowledge base** (e.g., documents, web pages) to find the most relevant chunks of information.  
2. **Augment:** These retrieved documents are added as **context** to the input of the language model.  
3. **Generate:** The LLM then uses both its internal knowledge and the retrieved context to produce a response.  

**Advantages of RAG:**  
- Reduces hallucinations (wrong answers) by grounding responses in real sources.  
- Keeps the model up-to-date without retraining.  
- Supports specialized domains like medical, legal, or academic knowledge.  

---

# Why combine OpenAI GPT with Pinecone vector DB for knowledge retrieval?

OpenAI GPT alone can generate fluent text, but it **cannot search through large external datasets efficiently**. To solve this:  

1. **Pinecone Vector Database** stores **embeddings** of documents (numerical representations of text).  
2. When a query is made, it is **converted into an embedding**.  
3. **Pinecone finds the most similar documents** using a similarity metric (e.g., cosine similarity).  
4. These retrieved documents are fed into GPT as **context**, enabling it to answer questions with **relevant external knowledge**.  

**Benefits of this combination:**  
- **Scalable retrieval:** Pinecone can store millions of documents efficiently.  
- **Dynamic knowledge:** GPT can provide natural language answers without needing to memorize every fact.  
- **Accurate responses:** Retrieval ensures the model’s answers are grounded in real data rather than only its training knowledge.  

---




# 2) Install Dependencies

In this step, we install all the Python libraries required to build our RAG Chatbot.  

### **Libraries and their purposes**

- **`langchain`** → A framework to build applications using Large Language Models (LLMs). It provides tools for chaining prompts, handling context, and integrating with external data sources.  

- **`langchain-openai`** → Provides connectors to OpenAI GPT models so that we can easily send prompts and receive responses from GPT.  

- **`langchain-pinecone`** → Integrates LangChain with the Pinecone vector database. It allows us to store, search, and retrieve vector embeddings efficiently for retrieval-augmented generation.  

- **`ipywidgets`** → Enables interactive user interfaces in Google Colab notebooks. We will use it to create input boxes, buttons, and display chat outputs dynamically.  

- **`tqdm`** → Provides progress bars for loops. This is helpful when embedding or indexing many documents so students can see progress in real-time.  

**Purpose:**  
Installing these packages ensures students understand not only **how to use them**, but also **why each is necessary** in a RAG workflow.  


In [ ]:
!pip install -q langchain langchain-openai langchain-pinecone langchain-community pinecone-client tqdm ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
!pip install -qU \
    langchain==0.3.23 \
    langchain-community==0.3.21 \
    langchain-pinecone==0.2.5 \
    langchain-openai==0.3.12 \
    datasets==3.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3

## 3) API Key Setup

In this step, we configure the API keys required to use OpenAI GPT models and Pinecone vector database.

### **Why API keys are needed**

- **OpenAI API key** → Allows access to OpenAI models (like GPT-4o-mini) to generate answers based on prompts.
- **Pinecone API key** → Provides access to Pinecone’s vector database for storing and retrieving embeddings of documents.

### **Security note**

- API keys are sensitive information.  
- We set them as **environment variables** (`os.environ`) so they are **not printed in the notebook output**.  
- This prevents accidental exposure if you share your notebook publicly.

### **How to use**

1. Replace the placeholder strings with your own API keys:
```python
OPENAI_API_KEY = "your_openai_api_key_here"
PINECONE_API_KEY = "your_pinecone_api_key_here"


In [ ]:
import os
# === Edit these two variables only ===
OPENAI_API_KEY = "your_openai_api_key_here"
PINECONE_API_KEY = "your_pinecone_api_key_here"

# Set environment variables (not printed)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

print("✅ API keys configured (not displayed).")


✅ API keys configured (not displayed).


## 4) Helper Functions: Load, Chunk, Initialize, Create Knowledge Base

In this section, we define the **core helper functions** that are essential for building a RAG Chatbot. Each function has a specific role in preparing and indexing knowledge from web URLs.

### **1. `load_web_content(urls)`**
- Scrapes web pages from a list of URLs.
- Returns a list of **Document objects**, each containing the text content and metadata.
- Metadata includes the **source URL**, which is important for traceability. This helps later show **where each answer comes from**.

### **2. `chunk_documents(documents)`**
- Splits long text documents into smaller chunks.
- Why? LLMs like GPT have a **maximum input token limit**, and smaller chunks improve the quality of retrieval and relevance.
- Parameters:
  - `chunk_size` → maximum number of characters per chunk.
  - `chunk_overlap` → number of characters overlapping between chunks to preserve context.

### **3. `initialize_pinecone_index(api_key)`**
- Creates a **vector index** in Pinecone if it doesn’t exist.
- The index stores **document embeddings**, enabling efficient similarity search.
- Cosine similarity is used to find the most relevant chunks for a query.

### **4. `create_knowledge_base(urls, api_key)`**
- Combines all steps to create a retrievable knowledge base:
  1. Load web content using `load_web_content`.
  2. Split documents into chunks using `chunk_documents`.
  3. Generate embeddings for each chunk using **OpenAI embeddings**.
  4. Store embeddings in **Pinecone vector DB**.


In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec, CloudProvider, AwsRegion, Metric
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.auto import tqdm

def load_web_content(urls):
    all_documents = []
    for url in urls:
        try:
            print(f"Loading content from: {url}")
            loader = WebBaseLoader(url)
            documents = loader.load()
            for doc in documents:
                doc.metadata['source'] = url
            all_documents.extend(documents)
            print(f"✓ Loaded {len(documents)} document(s) from {url}")
        except Exception as e:
            print(f"✗ Error loading {url}: {str(e)}")
    return all_documents

def chunk_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len)
    chunks = text_splitter.split_documents(documents)
    print(f"Split into {len(chunks)} chunks")
    return chunks

def initialize_pinecone_index(api_key, index_name="rag-pipeline"):
    pc = Pinecone(api_key=api_key)
    if not pc.has_index(name=index_name):
        pc.create_index(
            name=index_name,
            metric=Metric.COSINE,
            dimension=1536,
            spec=ServerlessSpec(cloud=CloudProvider.AWS, region=AwsRegion.US_EAST_1)
        )
        print(f"Created new index: {index_name}")
    else:
        print(f"Using existing index: {index_name}")
    return pc.Index(name=index_name)

def create_knowledge_base(urls, api_key, index_name="rag-pipeline"):
    documents = load_web_content(urls)
    if not documents:
        raise ValueError("No documents were loaded successfully")
    chunks = chunk_documents(documents)
    embed_model = OpenAIEmbeddings(model="text-embedding-3-small")
    index = initialize_pinecone_index(api_key, index_name)
    print("Embedding and indexing documents...")
    batch_size = 100
    for i in tqdm(range(0, len(chunks), batch_size)):
        i_end = min(len(chunks), i + batch_size)
        batch = chunks[i:i_end]
        ids = [f"doc-{i+j}" for j in range(len(batch))]
        texts = [doc.page_content for doc in batch]
        embeds = embed_model.embed_documents(texts)
        metadata = [{'text': doc.page_content, 'source': doc.metadata.get('source', 'unknown')} for doc in batch]
        index.upsert(vectors=zip(ids, embeds, metadata))
    print(f"✓ Indexed {len(chunks)} chunks")
    return PineconeVectorStore(index=index, embedding=embed_model, text_key="text")

def augment_prompt(query, vectorstore, k=3):
    results = vectorstore.similarity_search(query, k=k)
    source_knowledge = "\n\n".join([x.page_content for x in results])
    sources = [x.metadata.get('source', 'unknown') for x in results]
    augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""
    return augmented_prompt, sources


## 5) Create Knowledge Base (Run Once)

In this step, we **load, embed, and index all web pages** into Pinecone to create a knowledge base.  

### **Key Points for Students**

- **Indexing is expensive:**  
  - Embedding each chunk of text requires API calls to OpenAI.  
  - Uploading embeddings to Pinecone also takes time.  
  - Therefore, we **run this step only once**. After the knowledge base is created, we can reuse it for multiple queries.

- **Vectorstore creation:**  
  - The `create_knowledge_base(urls, api_key)` function returns a **vectorstore object**, which contains the embeddings and Pinecone index.  
  - This is stored in a variable named `vectorstore` for reuse.  
  - Example:  
    ```python
    vectorstore = create_knowledge_base(urls, api_key=PINECONE_API_KEY, index_name="rag-pipeline")
    ```
  - Once created, you can use `vectorstore` for **retrieval without re-indexing**.

- **Editing URLs:**  
  - Students can **add or remove web pages** in the `urls` list to change the knowledge base content.  
  - Example:  
    ```python
    urls = [
        "https://www.nwmissouri.edu/csis/msacs/",
        "https://www.nwmissouri.edu/csis/msacs/about.htm",
        # Add your own URLs here
    ]
    ```

**Purpose:**  
This step teaches students how **raw web content is converted into a retrievable knowledge base** and emphasizes the cost and time implications of embedding and indexing.


In [ ]:
# Example URLs (edit if needed)
urls = [
    "https://www.nwmissouri.edu/csis/msacs/",
    "https://www.nwmissouri.edu/csis/msacs/about.htm",
    "https://www.nwmissouri.edu/academics/graduate/masters/applied-computer-science.htm",
    "https://www.nwmissouri.edu/csis/msacs/apply/index.htm",
    "https://www.nwmissouri.edu/csis/msacs/courses.htm"
]

# Create knowledge base and store in `vectorstore` variable for reuse
print("Creating knowledge base from web URLs...")
vectorstore = create_knowledge_base(urls, api_key=os.environ.get("PINECONE_API_KEY"), index_name="rag-pipeline")
print('Vectorstore variable is ready for queries.')


Creating knowledge base from web URLs...
Loading content from: https://www.nwmissouri.edu/csis/msacs/
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/csis/msacs/
Loading content from: https://www.nwmissouri.edu/csis/msacs/about.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/csis/msacs/about.htm
Loading content from: https://www.nwmissouri.edu/academics/graduate/masters/applied-computer-science.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/academics/graduate/masters/applied-computer-science.htm
Loading content from: https://www.nwmissouri.edu/csis/msacs/apply/index.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/csis/msacs/apply/index.htm
Loading content from: https://www.nwmissouri.edu/csis/msacs/courses.htm
✓ Loaded 1 document(s) from https://www.nwmissouri.edu/csis/msacs/courses.htm
Split into 26 chunks
Using existing index: rag-pipeline
Embedding and indexing documents...


  0%|          | 0/1 [00:00<?, ?it/s]

✓ Indexed 26 chunks
Vectorstore variable is ready for queries.


## 6) Reusable Query Cell
### **How RAG retrieval works**

1. **Query:** Student enters a question.  
2. **Similarity search:** The `vectorstore.similarity_search(query, k=3)` retrieves the **top 3 most relevant document chunks** from the knowledge base.  
   - `k=3` means we consider **the 3 most similar chunks** for context.  
3. **Augment prompt:** These retrieved chunks are added to the prompt, providing the LLM with relevant context.  
4. **LLM generates answer:** GPT uses the augmented prompt to produce a response.  

### **Key teaching points**

- **Reusable:** This cell can be run **multiple times** for different queries without re-indexing the documents.  
- **Dynamic:** Each query retrieves fresh context from the vectorstore.  
- Students can **experiment with `k`** to see how increasing or decreasing the number of chunks affects answer quality.  


In [ ]:
# Example: change this query and re-run as many times as you want
query = "What courses are offered in the MSACS program?"

# Augment with context and call the chat model
augmented_query, sources = augment_prompt(query, vectorstore, k=3)

chat = ChatOpenAI(openai_api_key=os.environ.get("OPENAI_API_KEY"), model="gpt-4o-mini")
system_msg = SystemMessage(content="You are a helpful assistant that answers questions based on the provided context.")

response = chat([system_msg, HumanMessage(content=augmented_query)])
print("\n=== QUESTION ===")
print(query)
print("\n=== ANSWER ===")
print(response.content)
print("\n=== SOURCES ===")
print(", ".join(set(sources)))


/tmp/ipython-input-1331979542.py:10: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat([system_msg, HumanMessage(content=augmented_query)])



=== QUESTION ===
What courses are offered in the MSACS program?

=== ANSWER ===
The M.S. Applied Computer Science (MSACS) program at Northwest offers a variety of computer science courses. Each course is linked to a detailed syllabus that includes prerequisite information and the semesters they are offered. For specific courses, including popular electives, you can check the program advisor for the most up-to-date information or visit the detailed course descriptions page at the provided link: [MSACS Course Descriptions](https://www.nwmissouri.edu/csis/msacs/courses.htm).

=== SOURCES ===
https://www.nwmissouri.edu/csis/msacs/FAQs.htm, https://www.nwmissouri.edu/csis/msacs/about.htm, https://www.nwmissouri.edu/csis/msacs/courses.htm


## 7) Interactive Chat UI

### **UI Components**

- **Input box:** Where students type their questions.  
- **Buttons:**  
  - **Ask** → sends the query to the RAG Chatbot.  
  - **Clear** → clears the conversation history.  
  - **Quit** → stops the interaction.  
- **Conversation history list:** Stores and displays all previous user queries and GPT answers.  

### **How it works**

1. User enters a question in the input box.  
2. Upon clicking **Ask**, the query is sent through the **retrieval + augmentation process**.  
3. GPT generates an answer using the retrieved context.  
4. The question and answer are displayed dynamically in a **simple chat bubble interface** using HTML.  
5. Buttons allow students to **manage the conversation** interactively.  




In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create widgets
input_box = widgets.Text(placeholder='Type your question here...', description='Question:', layout=widgets.Layout(width='80%'))
ask_button = widgets.Button(description='Ask', button_style='primary')
quit_button = widgets.Button(description='Quit', button_style='danger')
clear_button = widgets.Button(description='Clear', button_style='warning')
output = widgets.HTML(value='', placeholder='', description='')

# Keep conversation history in a list
conversation_history = []

def render_history():
    # Render simple HTML chat window
    html = '<div style="max-height:400px; overflow:auto; border:1px solid #ddd; padding:10px; background:#f9f9f9;">'
    for turn in conversation_history:
        role = turn['role']
        text = turn['text'].replace('\n', '<br>')
        if role == 'user':
            html += f"<div style='text-align:right; margin:6px 0;'><b>You:</b> <span style='background:#dbeafe; padding:6px; border-radius:6px;'>{text}</span></div>"
        else:
            html += f"<div style='text-align:left; margin:6px 0;'><b>Bot:</b> <span style='background:#eef2ff; padding:6px; border-radius:6px;'>{text}</span></div>"
    html += '</div>'
    output.value = html

def on_ask_clicked(b):
    question = input_box.value.strip()
    if not question:
        return
    conversation_history.append({'role':'user','text':question})
    render_history()
    input_box.value = ''

    # Retrieve context and get answer
    augmented_query, sources = augment_prompt(question, vectorstore, k=3)
    chat = ChatOpenAI(openai_api_key=os.environ.get("OPENAI_API_KEY"), model="gpt-4o-mini")
    system_msg = SystemMessage(content="You are a helpful assistant that answers questions based on the provided context.")
    resp = chat([system_msg, HumanMessage(content=augmented_query)])
    answer = resp.content
    conversation_history.append({'role':'bot','text':answer + "<br><br><small><i>Sources: " + ", ".join(set(sources)) + "</i></small>"})
    render_history()

def on_quit_clicked(b):
    clear_output(wait=True)
    print("Chat UI closed. You can re-run the Chat UI cell to reopen it.")

def on_clear_clicked(b):
    conversation_history.clear()
    render_history()

ask_button.on_click(on_ask_clicked)
quit_button.on_click(on_quit_clicked)
clear_button.on_click(on_clear_clicked)

ui = widgets.HBox([input_box, ask_button, clear_button, quit_button])
display(ui, output)
render_history()


HTML(value='', placeholder='')